In [1]:
!pip install numpy
!pip install nltk
!pip install tensorflow
!pip install tflearn

In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import pickle
import json
with open('intents.json') as file:
    data=json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
'''
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
'''
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append (intent["tag"])


In [5]:
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [6]:
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

In [7]:
training=numpy.array(training)
output=numpy.array(output)

In [8]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.22801 | time: 0.001s
| Adam | epoch: 1500 | loss: 0.22801 - acc: 0.9873 -- iter: 08/10
Training Step: 3000  | total loss: 0.20582 | time: 0.003s
| Adam | epoch: 1500 | loss: 0.20582 - acc: 0.9886 -- iter: 10/10
--
INFO:tensorflow:C:\Users\user\BotTrial\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [11]:
def chat():
    print("Start talking with your bot (type quit to stop):")
    while True:
        inp=input("You:")
        if inp.lower()=="quit":
            break
        results=model.predict([bag_of_words(inp,words)])
        results_index=numpy.argmax(results)
        tag=labels[results_index]

        for tg in data["intents"]:
            if tg["tag"]==tag:
                responses=tg["responses"]
        print(random.choice(responses))
chat()

Start talking with your bot (type quit to stop):
You:Hey
Hi there, how can I help?
You:hello
Hi there, how can I help?
You:ty
Happy to help!
You:good
Any time!
You:bye
Bye!
You:quit
